In [1]:
import requests
import csv
import time
import json
import pickle

SERVER_URL = "http://34.71.138.79:9090"
ENDPOINT = "/sybil/{binary_or_affine}/{home_or_defense}"
URL = SERVER_URL + ENDPOINT

TEAM_TOKEN = "DdE56eC1EvRwWcuy"

In [ ]:
def sybil_attack(ids, home_or_defense: str, binary_or_affine: str = "binary"):
    if home_or_defense not in ["home", "defense"] or binary_or_affine not in ["binary", "affine"]:
        raise "Invalid endpoint"

    ids = ",".join(map(str, ids))
    url = URL.format(binary_or_affine=binary_or_affine, home_or_defense=home_or_defense)
    response = requests.get(
        url=url,
        params={"ids": ids},
        headers={"token": TEAM_TOKEN}
    )

    if response.status_code == 200:
        return response.json()["representations"]
    else:
        raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")

In [ ]:
COMMON_IDS = img_ids[:1000]

NameError: name 'img_ids' is not defined

In [ ]:
for i in range(1, 20):
    output = []

    print(i)
    ids = COMMON_IDS + img_ids[i*1000:(i+1)*1000]

    if i == 1:
        type_ = "home"
    else:
        type_ = "defense"

    print(type_, len(ids))
    response = sybil_attack(ids, type_)

    if i != 1:
        res = requests.post("http://34.71.138.79:9090/sybil/binary/reset/defense", headers={"token": TEAM_TOKEN})
        print("reset: ", res.status_code, res.content)
        time.sleep(3)
        res = requests.post("http://34.71.138.79:9090/sybil/binary/reset/defense", headers={"token": TEAM_TOKEN})
        print("reset: ", res.status_code, res.content)
        time.sleep(3)

    for idx in range(1000):
        img_id = img_ids[idx]
        output.append({"type": type_, "img_id": img_id, "y": response[idx]})

    x = 1000
    for idx in range(i*1000, (i+1)*1000):
        img_id = img_ids[idx]
        output.append({"type": type_, "img_id": img_id, "y": response[x]})
        x += 1

    print("saving")
    with open(f"OUT/{type_}{i}.obj", "wb") as f:
        pickle.dump(output, f)

18
defense 2000
reset:  200 b'{"msg":"Successful sybil binary reset defense"}'
reset:  200 b'{"msg":"Successful sybil binary reset defense"}'
saving
19
defense 2000
reset:  200 b'{"msg":"Successful sybil binary reset defense"}'
reset:  200 b'{"msg":"Successful sybil binary reset defense"}'
saving


In [18]:
import torch, torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import numpy as np
import pandas as pd
import torch.utils.data as data
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json
import pickle

In [58]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [66]:
class MappingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(384, 2048)
        self.bn1 = nn.BatchNorm1d(2048)
        self.act1 = nn.ReLU()
        self.lin2 = nn.Linear(2048, 1024)
        self.bn2 = nn.BatchNorm1d(1024)
        self.act2 = nn.ReLU()
        self.lin3 = nn.Linear(1024, 512)
        self.bn3 = nn.BatchNorm1d(512)
        self.act3 = nn.ReLU()
        self.lin4 = nn.Linear(512, 384)

    def forward(self, x):
        x = self.lin1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.lin3(x)
        x = self.bn3(x)
        x = self.act3(x)
        x = self.lin4(x)
        return x

In [67]:
device = torch.device("cuda")

In [69]:
for defense_i in range(2, 20):
  print("Training defense ", defense_i)
  with open("home1.obj", "rb") as f:
    home1 = pickle.load(f)

  with open(f"defense{defense_i}.obj", "rb") as f:
    defense = pickle.load(f)

  net = MappingNet()
  net.to(device)
  loss_fn = nn.MSELoss()
  optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

  X = [v["y"] for v in defense[:1000]]
  Y = [v["y"] for v in home1[:1000]]

  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.001)

  train_dataset = torch.utils.data.TensorDataset(torch.tensor(x_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float))
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

  net.train()

  for epoch in range(201):
      for data_inputs, data_labels in train_loader:

          data_inputs = data_inputs.to(device)
          data_labels = data_labels.to(device)
          preds = net(data_inputs)
          preds = preds.squeeze(dim=1)

          loss = loss_fn(preds, data_labels.float())

          optimizer.zero_grad()

          loss.backward()

          optimizer.step()

      if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")


  net.eval()

  x_test_tensor = torch.tensor([v["y"] for v in defense[1000:]], dtype=torch.float).to(device)

  res = net(x_test_tensor).tolist()

  out = []
  for idx in range(1000, 2000):
    out.append({"img_id": defense[idx]["img_id"], "y": res[idx-1000]})

  with open(f"res{defense_i}.json", "w") as f:
    json.dump(out, f)


Training defense  2
Epoch: 0, loss: 0.161
Epoch: 10, loss: 0.0747
Epoch: 20, loss: 0.0557
Epoch: 30, loss: 0.0439
Epoch: 40, loss: 0.0396
Epoch: 50, loss: 0.0533
Epoch: 60, loss: 0.0521
Epoch: 70, loss: 0.0371
Epoch: 80, loss: 0.0426
Epoch: 90, loss: 0.0555
Epoch: 100, loss: 0.0288
Epoch: 110, loss: 0.0309
Epoch: 120, loss: 0.0447
Epoch: 130, loss: 0.0436
Epoch: 140, loss: 0.0288
Epoch: 150, loss: 0.0296
Epoch: 160, loss: 0.0311
Epoch: 170, loss: 0.0388
Epoch: 180, loss: 0.0456
Epoch: 190, loss: 0.0377
Epoch: 200, loss: 0.0359
Training defense  3
Epoch: 0, loss: 0.0894
Epoch: 10, loss: 0.0855
Epoch: 20, loss: 0.0584
Epoch: 30, loss: 0.0622
Epoch: 40, loss: 0.0656
Epoch: 50, loss: 0.0516
Epoch: 60, loss: 0.0536
Epoch: 70, loss: 0.0762
Epoch: 80, loss: 0.0318
Epoch: 90, loss: 0.0502
Epoch: 100, loss: 0.0308
Epoch: 110, loss: 0.0573
Epoch: 120, loss: 0.0355
Epoch: 130, loss: 0.0281
Epoch: 140, loss: 0.0326
Epoch: 150, loss: 0.038
Epoch: 160, loss: 0.0342
Epoch: 170, loss: 0.028
Epoch: 180

In [70]:
with open("home1.obj", "rb") as f:
  home1 = pickle.load(f)


  out = []
  for idx in range(2000):
    out.append({"img_id": home1[idx]["img_id"], "y": home1[idx]["y"]})

  with open(f"res1.json", "w") as f:
    json.dump(out, f)


In [71]:
ids = []
results = []

for i in range(1, 20):
  with open(f"res{i}.json") as f:
    data = json.load(f)

    for row in data:
      ids.append(row["img_id"])
      results.append(row["y"])



In [72]:
np.savez(
        "sybil_submission3.npz",
        ids=ids,
        representations=results,
    )

In [56]:
import requests

In [73]:

with open("sybil_submission3.npz", "rb") as f:
    response = requests.post(
        "http://34.71.138.79:9090/sybil/binary/submit", files={"file": f}, headers={"token": "DdE56eC1EvRwWcuy"}
    )

    print(response.status_code, response.content)

200 b'{"score":12.786150862676854}'


In [ ]:
home1[0]

{'type': 'home',
 'img_id': 101031,
 'y': [1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
